In [ ]:
import sys
sys.path.append("../src")
from bigbrotr import Bigbrotr
from event import Event
from relay import Relay
from relay_metadata import RelayMetadata
import utils
import pandas as pd

# Utils

In [ ]:
private_key, public_key = utils.generate_nostr_keypair()
e = utils.generate_event(private_key, public_key, 1, [], "test")
assert utils.verify_sig(e['id'], e['pubkey'], e['sig']), "Signature verification failed"
assert utils.calc_event_id(e['pubkey'], e['created_at'], e['kind'], e['tags'], e['content']) == e['id'], "Event ID calculation failed"
e = Event(e['id'], e['pubkey'], e['created_at'], e['kind'], e['tags'], e['content'], e['sig'])
e

# Database

In [ ]:
bigbrotr = Bigbrotr(
    host="localhost",
    port=5432,
    user="admin",
    password="admin",  
    dbname="bigbrotr"
)

In [ ]:
bigbrotr.connect()

In [ ]:
query = """
SELECT DISTINCT relay_url
FROM events_relays
WHERE seen_at >= %s
"""
import time
now = int(time.time()) - 120
df = pd.read_sql(query, bigbrotr.conn, params=(now,))
df

In [ ]:
import psycopg2

def bytes_to_gb(b):
    return b / (1024 ** 3)

conn = psycopg2.connect(
    dbname="bigbrotr",
    user="admin",
    password="admin",
    host="localhost",
    port=5432
)
cursor = conn.cursor()

# Recupera tutte le tabelle nel tuo schema pubblico
cursor.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public' AND table_type = 'BASE TABLE'
""")
tables = [row[0] for row in cursor.fetchall()]

print("📊 Analisi dello spazio per tabella:\n")

for table in tables:
    print(f"🧾 TABELLA: {table}")
    
    # Numero righe
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    total_rows = cursor.fetchone()[0]
    print(f"Numero righe: {total_rows}")

    if total_rows == 0:
        print(" (Tabella vuota)\n")
        continue

    # Calcola dimensione media per colonna
    cursor.execute(f"""
        SELECT string_agg(
            'AVG(pg_column_size(\"' || column_name || '\")) AS \"' || column_name || '\"',
            ', '
        )
        FROM information_schema.columns
        WHERE table_name = %s AND table_schema = 'public'
    """, (table,))
    agg_expr = cursor.fetchone()[0]

    avg_size_query = f"SELECT {agg_expr} FROM {table}"
    cursor.execute(avg_size_query)
    avg_sizes = cursor.fetchone()
    colnames = [desc.name for desc in cursor.description]

    print("{:<20} {:>12} {:>12}".format("Colonna", "Avg (bytes)", "Totale (GB)"))
    total_data_bytes = 0

    for i, col in enumerate(colnames):
        avg_bytes = float(avg_sizes[i])
        total_bytes = avg_bytes * total_rows
        total_data_bytes += total_bytes
        print("{:<20} {:>12.2f} {:>12.2f}".format(col, avg_bytes, bytes_to_gb(total_bytes)))

    print(f"Totale dati stimati: {bytes_to_gb(total_data_bytes):.2f} GB")

    # Recupera indici della tabella con dimensione
    cursor.execute(f"""
        SELECT 
            indexname, 
            pg_relation_size(indexrelid) AS size_bytes
        FROM pg_indexes
        JOIN pg_class ON pg_class.relname = indexname
        JOIN pg_index ON pg_class.oid = pg_index.indexrelid
        WHERE tablename = %s
    """, (table,))
    index_data = cursor.fetchall()

    total_index_bytes = 0
    print("\n📦 Indici:")
    print("{:<30} {:>12}".format("Indice", "Size (GB)"))
    for index_name, size_bytes in index_data:
        total_index_bytes += size_bytes
        print("{:<30} {:>12.2f}".format(index_name, bytes_to_gb(size_bytes)))

    print(f"Totale indici: {bytes_to_gb(total_index_bytes):.2f} GB")
    print(f"Totale complessivo stimato: {bytes_to_gb(total_data_bytes + total_index_bytes):.2f} GB\n")
    print("─" * 60)

cursor.close()
conn.close()

In [ ]:
query = """
SELECT 
    relay_url,
    COUNT(*) AS event_count
FROM 
    events_relays
GROUP BY 
    relay_url
ORDER BY 
    event_count ASC;
"""
df = pd.read_sql(query, bigbrotr.conn)
df

In [ ]:
# plot cdf fraction of relays vs number of events
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 6))
plt.plot(df['event_count'].value_counts().sort_index().cumsum() / df['event_count'].count(), marker='o')
plt.title('CDF of Relays vs Number of Events')
plt.xlabel('Number of Events')
plt.ylabel('Cumulative Fraction of Relays')
plt.xticks(rotation=45)
# plt.xscale('log')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
query = "SELECT COUNT(*) FROM events WHERE kind = 1"
bigbrotr.execute(query)
rows = bigbrotr.fetchall()
print(f"Number of events in the database: {rows[0][0]}")

In [ ]:
query = """
SELECT relay_url, COUNT(*) AS count
FROM relay_metadata
GROUP BY relay_url
ORDER BY count DESC
"""

bigbrotr.execute(query)
rows = bigbrotr.fetchall()

# Create DataFrame
df = pd.DataFrame(rows, columns=['relay_url', 'count'])
df

In [ ]:
df.hist(column='count', bins=50, edgecolor='black')

In [ ]:
query = '''
SELECT DISTINCT ON (relay_url)
    relay_url,
    event_id
FROM events_relays
ORDER BY relay_url, seen_at DESC;
'''
bigbrotr.execute(query)
rows = bigbrotr.fetchall()
df = pd.DataFrame(rows, columns=['relay_url', 'event_id'])

In [ ]:
df

In [ ]:
df['year_month'] = pd.to_datetime(df['max_created_at'], unit='s').dt.to_period('M')

In [ ]:
df.value_counts('year_month').sort_index().plot(kind='bar', figsize=(12, 6)) 

In [ ]:
query = """
SELECT COUNT(*)
FROM events e
WHERE NOT EXISTS (
    SELECT 1
    FROM events_relays er
    WHERE er.event_id = e.id
);
"""
bigbrotr.execute(query)
rows = bigbrotr.fetchall()
print(f"Number of events without relays: {rows[0][0]}")

In [ ]:
query = '''
SELECT COUNT(*)
FROM events_relays
WHERE relay_url = %s
'''
args = ('wss://relay.nostr.band',)
bigbrotr.execute(query, args)
rows = bigbrotr.fetchall()
rows

In [ ]:
query = '''
SELECT event_id
FROM events_relays
WHERE relay_url = %s
'''
args = ('wss://relay.nostr.band',)
bigbrotr.execute(query, args)
rows = bigbrotr.fetchall()
rows = pd.DataFrame(rows, columns=['event_id'])
event_ids = rows['event_id'].tolist()

query = '''
SELECT MAX(created_at)
FROM events
WHERE id = ANY(%s)
'''
args = (event_ids,)
bigbrotr.execute(query, args)
rows = bigbrotr.fetchall()
rows

In [ ]:
# bigbrotr.conn.autocommit = True
# bigbrotr.execute('VACUUM FULL events;')
# bigbrotr.execute('VACUUM FULL events_relays;')
# bigbrotr.execute('VACUUM FULL relay_metadata;')
# bigbrotr.execute('VACUUM FULL relays;')

In [ ]:
bigbrotr.close()